# Processing

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC"
}

swaped_states = {value:key for key, value in us_state_to_abbrev.items()}

## Loading datasets

In [33]:
shootings = pd.read_csv('./datasets/shootings2.csv')
gun_possesion = pd.read_csv('./datasets/GunsOwnerShip_dataset.csv')
hdi = pd.read_csv('./datasets/HDI.csv')
income_index = pd.read_csv('./datasets/GDL-Income-index-data.csv')
educational_index = pd.read_csv('./datasets/GDL-Educational-index--data.csv')

shootings['date'] = pd.to_datetime(shootings['date'])

### Concat gun-ownership

In [34]:
def ownership_value(state_tag):
    """ Para cada estado devuelve el valor de posesion de armas de este"""
    try:
        state_name = swaped_states[state_tag]
        return gun_possesion[gun_possesion["State"] == state_name]["gunOwnership"].to_list()[0]
    except:
        return np.nan
    
shooting_v1 = shootings.copy()
shooting_v1["gun-ownership"] = shooting_v1.apply(lambda row : ownership_value(row["state"]), axis=1)
shooting_v1.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,arms_category,gun-ownership
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,Asian,Shelton,WA,True,attack,Not fleeing,False,Guns,NaN
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,White,Aloha,OR,False,attack,Not fleeing,False,Guns,NaN
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,Hispanic,Wichita,KS,False,other,Not fleeing,False,Unarmed,NaN
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,White,San Francisco,CA,True,attack,Not fleeing,False,Other unusual objects,NaN
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,Hispanic,Evans,CO,False,attack,Not fleeing,False,Piercing objects,NaN


In [35]:
shooting_v1.to_csv('./datasets/shootings_with_gun_ownership.csv', index=False)

### Concat HDI

In [36]:
selected_cols = ['Region', '2015', '2016', '2017', '2018', '2019']
hdi_v1 = hdi[selected_cols]

In [37]:
hdi_v1.rename(columns={col: 'hdi_' + col for col in hdi_v1.columns if col in selected_cols[1::]}, inplace=True)
hdi_v1.head()

,Region,hdi_2015,hdi_2016,hdi_2017,hdi_2018,hdi_2019
0,Total,0.921,0.922,0.924,0.926,0.926
1,Alabama,0.880,0.882,0.884,0.885,0.886
2,Alaska,0.934,0.935,0.936,0.936,0.936
3,Arizona,0.908,0.909,0.911,0.912,0.913
4,Arkansas,0.881,0.882,0.884,0.885,0.886


In [38]:
def value_hdi(row):
    anno = row['date'].year
    if anno != 2020:
        state = row["state"]
        state_name = swaped_states[state]
        return hdi_v1[hdi_v1['Region'] == state_name]['hdi_' + str(anno)].to_list()[0]
    return np.nan

shooting_v1["hdi"] = shooting_v1.apply(lambda row : value_hdi(row), axis=1)
shooting_v1.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,arms_category,gun-ownership,hdi
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,Asian,Shelton,WA,True,attack,Not fleeing,False,Guns,NaN,0.940
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,White,Aloha,OR,False,attack,Not fleeing,False,Guns,NaN,0.930
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,Hispanic,Wichita,KS,False,other,Not fleeing,False,Unarmed,NaN,0.922
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,White,San Francisco,CA,True,attack,Not fleeing,False,Other unusual objects,NaN,0.930
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,Hispanic,Evans,CO,False,attack,Not fleeing,False,Piercing objects,NaN,0.942


In [46]:
shooting_v1.to_csv('./datasets/shootings_with_gun_ownership_&_hdi.csv', index=False)

### Concat Income Index

In [39]:
selected_cols = ['Region', '2015', '2016', '2017', '2018', '2019'] # take only desired columns
income_index = income_index[selected_cols]
income_index.head()

,Region,2015,2016,2017,2018,2019
0,Total,0.965,0.966,0.969,0.973,0.976
1,Alabama,0.918,0.920,0.922,0.926,0.929
2,Alaska,1.000,1.000,1.000,1.000,1.000
3,Arizona,0.924,0.925,0.927,0.931,0.934
4,Arkansas,0.915,0.916,0.918,0.922,0.925


In [40]:
# Rename year columns with description
income_index.rename(columns={col: 'income_' + col for col in income_index.columns if col in selected_cols[1::]}, inplace=True)
income_index.head()

,Region,income_2015,income_2016,income_2017,income_2018,income_2019
0,Total,0.965,0.966,0.969,0.973,0.976
1,Alabama,0.918,0.920,0.922,0.926,0.929
2,Alaska,1.000,1.000,1.000,1.000,1.000
3,Arizona,0.924,0.925,0.927,0.931,0.934
4,Arkansas,0.915,0.916,0.918,0.922,0.925


In [41]:
def value_income(row):
    row_year = row['date'].year
    if row_year != 2020:
        state = row["state"]
        state_name = swaped_states[state]
        return income_index[income_index['Region'] == state_name]['income_' + str(row_year)].to_list()[0]
    return np.nan

shooting_v1["income_index"] = shooting_v1.apply(lambda row : value_income(row), axis=1)
shooting_v1.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,arms_category,gun-ownership,hdi,income_index
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,Asian,Shelton,WA,True,attack,Not fleeing,False,Guns,NaN,0.940,0.981
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,White,Aloha,OR,False,attack,Not fleeing,False,Guns,NaN,0.930,0.964
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,Hispanic,Wichita,KS,False,other,Not fleeing,False,Unarmed,NaN,0.922,0.955
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,White,San Francisco,CA,True,attack,Not fleeing,False,Other unusual objects,NaN,0.930,0.986
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,Hispanic,Evans,CO,False,attack,Not fleeing,False,Piercing objects,NaN,0.942,0.973


In [47]:
shooting_v1.to_csv('./datasets/shootings_with_gun_ownership_&_hdi_&_income.csv', index=False)

### Concat Educational Index

In [42]:
selected_cols = ['Region', '2015', '2016', '2017', '2018', '2019'] # take only desired columns
educational_index = educational_index[selected_cols]
educational_index.head()

,Region,2015,2016,2017,2018,2019
0,Total,0.893,0.896,0.899,0.899,0.899
1,Alabama,0.872,0.875,0.878,0.878,0.878
2,Alaska,0.908,0.911,0.914,0.914,0.914
3,Arizona,0.883,0.886,0.889,0.889,0.889
4,Arkansas,0.866,0.869,0.872,0.872,0.872


In [43]:
# Rename year columns with description
educational_index.rename(columns={col: 'educational_' + col for col in educational_index.columns if col in selected_cols[1::]}, inplace=True)
educational_index.head()

,Region,educational_2015,educational_2016,educational_2017,educational_2018,educational_2019
0,Total,0.893,0.896,0.899,0.899,0.899
1,Alabama,0.872,0.875,0.878,0.878,0.878
2,Alaska,0.908,0.911,0.914,0.914,0.914
3,Arizona,0.883,0.886,0.889,0.889,0.889
4,Arkansas,0.866,0.869,0.872,0.872,0.872


In [51]:
def value_educational(row):
    row_year = row['date'].year
    if row_year != 2020:
        state = row["state"]
        state_name = swaped_states[state]
        return educational_index[educational_index['Region'] == state_name]['educational_' + str(row_year)].to_list()[0]
    return np.nan

shooting_v1["educational_index"] = shooting_v1.apply(lambda row : value_educational(row), axis=1)
shooting_v1.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,arms_category,gun-ownership,hdi,income_index,educational_index
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,Asian,Shelton,WA,True,attack,Not fleeing,False,Guns,NaN,0.940,0.981,0.918
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,White,Aloha,OR,False,attack,Not fleeing,False,Guns,NaN,0.930,0.964,0.911
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,Hispanic,Wichita,KS,False,other,Not fleeing,False,Unarmed,NaN,0.922,0.955,0.909
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,White,San Francisco,CA,True,attack,Not fleeing,False,Other unusual objects,NaN,0.930,0.986,0.872
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,Hispanic,Evans,CO,False,attack,Not fleeing,False,Piercing objects,NaN,0.942,0.973,0.932


In [52]:
shooting_v1.to_csv('./datasets/shootings_with_gun_ownership_&_hdi_&_income_&_educational.csv', index=False)

In [53]:
shooting_v1.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,arms_category,gun-ownership,hdi,income_index,educational_index
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,Asian,Shelton,WA,True,attack,Not fleeing,False,Guns,NaN,0.940,0.981,0.918
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,White,Aloha,OR,False,attack,Not fleeing,False,Guns,NaN,0.930,0.964,0.911
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,Hispanic,Wichita,KS,False,other,Not fleeing,False,Unarmed,NaN,0.922,0.955,0.909
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,White,San Francisco,CA,True,attack,Not fleeing,False,Other unusual objects,NaN,0.930,0.986,0.872
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,Hispanic,Evans,CO,False,attack,Not fleeing,False,Piercing objects,NaN,0.942,0.973,0.932


In [54]:
shooting_v1.columns

Index(['id', 'name', 'date', 'manner_of_death', 'armed', 'age', 'gender',
       'race', 'city', 'state', 'signs_of_mental_illness', 'threat_level',
       'flee', 'body_camera', 'arms_category', 'gun-ownership', 'hdi',
       'income_index', 'educational_index'],
      dtype='object')